In [8]:
import requests
from bs4 import BeautifulSoup

url = "https://www.cnn.com"
response = requests.get(url)
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, 'lxml')

headings = soup.find_all(['h1', 'h2'])
title_candidates = [(h.text.strip(), len(h.text.strip())) for h in headings if h.text.strip()]
longest_title = max(title_candidates, key=lambda x: x[1])[0] if title_candidates else "No title found"
print("Extracted Title:", longest_title)

paragraphs = soup.find_all('p')
body_candidates = [p.text.strip() for p in paragraphs if len(p.text.strip()) > 100]
article_body = ' '.join(body_candidates) if body_candidates else "No body found"
print("Extracted Body Snippet:", article_body[:200] + "...")

Extracted Title: Manhunt continues for Charlie Kirk's killer as rifle believed to be used in shooting found
Extracted Body Snippet: © 2025 Cable News Network. A Warner Bros. Discovery Company. All Rights Reserved.  CNN Sans ™ & © 2016 Cable News Network....


In [2]:
import requests
from bs4 import BeautifulSoup

url = "https://www.rt.com"
response = requests.get(url)
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, 'lxml')

headings = soup.find_all(['h1', 'h2'])
title_candidates = [(h.text.strip(), len(h.text.strip())) for h in headings if h.text.strip()]
longest_title = max(title_candidates, key=lambda x: x[1])[0] if title_candidates else "No title found"
print("Extracted Title:", longest_title)

paragraphs = soup.find_all('p')
body_candidates = [p.text.strip() for p in paragraphs if len(p.text.strip()) > 100]
article_body = ' '.join(body_candidates) if body_candidates else "No body found"
print("Extracted Body Snippet:", article_body[:200] + "...")

Extracted Title: Top stories
Extracted Body Snippet: No body found...


In [3]:
import requests
from bs4 import BeautifulSoup

url = "https://www.rbc.ru/"
response = requests.get(url)
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, 'lxml')

headings = soup.find_all(['h1', 'h2'])
title_candidates = [(h.text.strip(), len(h.text.strip())) for h in headings if h.text.strip()]
longest_title = max(title_candidates, key=lambda x: x[1])[0] if title_candidates else "No title found"
print("Extracted Title:", longest_title)

paragraphs = soup.find_all('p')
body_candidates = [p.text.strip() for p in paragraphs if len(p.text.strip()) > 100]
article_body = ' '.join(body_candidates) if body_candidates else "No body found"
print("Extracted Body Snippet:", article_body[:200] + "...")

Extracted Title: Новости
Extracted Body Snippet: No body found...


In [4]:
import requests
from bs4 import BeautifulSoup

url = "https://www.aljazeera.net/"
response = requests.get(url)
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, 'lxml')

headings = soup.find_all(['h1', 'h2'])
title_candidates = [(h.text.strip(), len(h.text.strip())) for h in headings if h.text.strip()]
longest_title = max(title_candidates, key=lambda x: x[1])[0] if title_candidates else "No title found"
print("Extracted Title:", longest_title)

paragraphs = soup.find_all('p')
body_candidates = [p.text.strip() for p in paragraphs if len(p.text.strip()) > 100]
article_body = ' '.join(body_candidates) if body_candidates else "No body found"
print("Extracted Body Snippet:", article_body[:200] + "...")

Extracted Title: يتصدر الآن
Extracted Body Snippet: أفادت وكالة الأنباء القطرية بأن الدوحة ستستضيف قمة عربية إسلامية طارئة يومي الأحد والاثنين المقبلين- لبحث الهجوم الإسرائيلي على عاصمة قطر. أفادت وكالة الأنباء القطرية بأن الدوحة ستستضيف قمة عربية إسلا...


In [6]:
import requests
from bs4 import BeautifulSoup

url = "https://www.bbc.com/"
response = requests.get(url)
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, 'lxml')

headings = soup.find_all(['h1', 'h2'])
title_candidates = [(h.text.strip(), len(h.text.strip())) for h in headings if h.text.strip()]
longest_title = max(title_candidates, key=lambda x: x[1])[0] if title_candidates else "No title found"
print("Extracted Title:", longest_title)

paragraphs = soup.find_all('p')
body_candidates = [p.text.strip() for p in paragraphs if len(p.text.strip()) > 100]
article_body = ' '.join(body_candidates) if body_candidates else "No body found"
print("Extracted Body Snippet:", article_body[:200] + "...")

Extracted Title: County Championship day four - Notts hold nerve for win, Leics seek promotion, radio & text
Extracted Body Snippet: Witnesses describe screaming and running to flee the scene after hearing the fatal gunshot that killed Kirk. Charlie Kirk's killing is another episode of gun violence in America and the latest in a li...


In [7]:
import requests
from bs4 import BeautifulSoup

url = "https://www.cnet.com/"
response = requests.get(url)
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, 'lxml')

headings = soup.find_all(['h1', 'h2'])
title_candidates = [(h.text.strip(), len(h.text.strip())) for h in headings if h.text.strip()]
longest_title = max(title_candidates, key=lambda x: x[1])[0] if title_candidates else "No title found"
print("Extracted Title:", longest_title)

paragraphs = soup.find_all('p')
body_candidates = [p.text.strip() for p in paragraphs if len(p.text.strip()) > 100]
article_body = ' '.join(body_candidates) if body_candidates else "No body found"
print("Extracted Body Snippet:", article_body[:200] + "...")

Extracted Title: Daily Puzzle Answers
Extracted Body Snippet: Our shopping experts unearth the best tech and home essential deals every day. If you make a purchase using our links, CNET may earn a commission. From gaming and the camera to new AI skills and the b...
